# IEL DATA EXPLORATION OLD
This Jupiter notebook is used to explore the data and record my thought process of how I will approach the problem. The first step is to import the necessary libraries, and then load the data. This notebook uses the data from multiple seperate sources meaning the results aren't as good as the new notebook. However,this notebook is still useful as it shows the thought process behind the data exploration.

In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import os

### Read in the data
Currently we are using data from the "Carradale Water at Dippen" Water Station. This is a water station which is located in Cambeltown, on the West Coast of Scotland. The reason for choosing this station is because it has approximately the same latitude, and geography as the fictitious island of Claddach and monitors a river pretty much all the way from the source to the river estuary. This means that the data should be representative of the data we would expect to see on Claddach.

The data comes from 4 seperate CSV files which are accessed from the following sources:
- National River Flow Archive (https://nrfa.ceh.ac.uk/data/station/spatial/88001); This is used to get the gauged daily flow and the catchment daily rainfall data.
- River Levels UK (https://riverlevels.uk/carradale-water-dippen#.Y8e3-9LP1hE); This is used to get the daily river level data.
- Scottish Environmental Protection Agency (https://www2.sepa.org.uk/waterlevels/?sd=t&lc=133077); This is used to get recent level data in more detail (quater-hourly opposed to daily, however only for the last 5 days.)

If we decide in the future to model our data using a different water station, we need to ensure that it records all 4 of the CSV files which are mentioned above. Before reading in the data, we took the following steps to format the data so that it is easier to read in and work with:
- Downloaded, renamed, and moved the CSV files to the data folder.
- Removed the first few rows of the CSV files which contained information about the data, and not the data itself.

Improvements which could be made in the future:
- Instead of using just one water station, we could aggregate data from multiple water stations to get a more representative sample of the data.
- Find quater-hourly data for the rainfall and flow data, as this could then be used with the SEPA data to get a more short-term view of the data.
- Find data for extreme weather events, such as floods, to see how the data changes during these events.
- Currently the data time frames have very little overlap (only 2017) which means despite having some data dating back to 1960s we can only use a small portion of it.

In [2]:
# Use numpy to read the 4 csv files into 4 seperate numpy arrays
flow = pd.read_csv('Data/NATIONAL_RIVER_FLOW_ARCHIVE_GAUGED_DAILY_FLOW.csv', error_bad_lines=False) # When reading the csv file, ignore the bad lines
rain = pd.read_csv('Data/NATIONAL_RIVER_FLOW_ARCHIVE_CATCHMENT_DAILY_RAINFALL.csv')
level = pd.read_csv('Data/RIVER_LEVELS_UK.csv')
level_recent = pd.read_csv('Data/SEPA_LEVEL_DATA.csv')


/tmp/ipykernel_26157/4235383111.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  flow = pd.read_csv('Data/NATIONAL_RIVER_FLOW_ARCHIVE_GAUGED_DAILY_FLOW.csv', error_bad_lines=False) # When reading the csv file, ignore the bad lines


FileNotFoundError: [Errno 2] No such file or directory: 'Data/NATIONAL_RIVER_FLOW_ARCHIVE_GAUGED_DAILY_FLOW.csv'

In [ ]:
# Merge the 3 datasets into one - excluding the level_recent dataset as it uses dataTime opposed to just date.
merged = pd.merge(flow, rain, on=['date'])
merged = pd.merge(merged, level, on=['date'])
merged.dropna(inplace=True)
merged = merged.drop('ignore', axis=1)
merged.describe()

# Find the number of days with more than 1mm of rain
merged[merged['cdr(mm)'] > 1].count()

### Plotting the data
First stage of exploring the data is to create a basic plot to see what the interactions are between the flow, rainfall and river level data. After this, we can start to look at the data in more detail.

In [ ]:
plt.plot(merged['date'], [i * 50 for i in merged['avg_level']])
#plt.plot(merged['date'], merged['gdf(m3/s)'])
plt.plot(merged['date'], merged['cdr(mm)'])
plt.legend(['Level', 'Rainfall'])
plt.show()

In [ ]:
# Sort the data by the average rainfall (independent variable) and plot the average flow and level (dependent variables)
byRainfall = merged.sort_values(by=['cdr(mm)'])
plt.plot(byRainfall['cdr(mm)'], byRainfall['gdf(m3/s)'], 'o')

In [ ]:
plt.plot(byRainfall['cdr(mm)'], byRainfall['avg_level'], 'o')

In [ ]:
# Attempt to plot average level against average flow
plt.plot(byRainfall['gdf(m3/s)'], byRainfall['avg_level'], 'o')

In [ ]:
# Plot all 3 variables on a 3D graph
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(byRainfall['gdf(m3/s)'], byRainfall['avg_level'], byRainfall['cdr(mm)'], c=byRainfall['cdr(mm)'], cmap='Greens');
ax.set_xlabel('Flow')
ax.set_ylabel('Level')
ax.set_zlabel('Rainfall')


### Simple Calculations

- The Catchment Area of the Carradale Water at Dippen Water Station is 58.5km^2 (https://www2.sepa.org.uk/waterlevels/default.aspx?sd=t&lc=133077).
- 1mm of rainfall is equal to 1 litre of water per square metre.
- hence the amount of water going into our river (measured in litres) on a given day is equal to: rainfall * 58.5 * 1,000,000 (to convert from km^2 to m^2)
- = rainfall * 58,500,000

- For the amount of water going out of the river (measured in litres) on a given day, we can use the flow data.
- We know that the flow data is measured in m^3/s, so we can convert this to litres (by multiplying by 1000) and then multiply by the number of seconds in a day (86400) to get the litres of water going out of the river on a given day.
- = flow * 1000 * 86400
- = flow * 86,400,000

Hence the water difference on a given day is equal to:
- = rainfall * 58,500,000 - flow * 86,400,000

Let's add this to our dataframe and plot it.


In [ ]:
waterDifference = [merged['cdr(mm)'][i]*58500000 - merged['gdf(m3/s)'][i]*86400000 for i in range(len(merged['cdr(mm)']))]
cumulativeWaterDifference = [sum(waterDifference[:i]) for i in range(len(waterDifference))]

print("minimum cumulative water difference:", min(cumulativeWaterDifference))
print("maximum cumulative water difference:",max(cumulativeWaterDifference))
print("mean cumulative water difference:",sum(cumulativeWaterDifference)/len(cumulativeWaterDifference))

As can be seen from the above numbers, we seem to be gaining a lot more water than we are losing. This is likely because we made two assumptions when calculating the water difference:
- We assumed that all of the rainfall was eventually going into the river.
- We didn't take avaperation into account.

The Scottish Governments states that the estimated percentage of precipitation which runoffs into the see is approximately 73% (https://www.gov.scot/publications/scotlands-marine-atlas-information-national-marine-plan/pages/7/) However, this is extremely variable from location to location (eg. in the USA this is as low as 30% (https://www.usgs.gov/special-topics/water-science-school/science/rain-and-precipitation). As well as the fact that on a daily, or even hourly basis, this will depend upon the weather conditions as more sunshine and warmer weather will lead to more evaporation and less ending up in the river. Therefore for now, we will set the percentage of rainfall which ends up in the river to a figure which results in the cumulative water difference throughout the year being equal to 0. This will likely be changed later to a more realistic value which changes throughout the year to account for the weather conditions.

In [ ]:
# To find the percentage that makes the culmulative water difference the closest to 0 we will use a binary search algorithm, starting with a percentage of 0.5.
percentage = 0.5
scale = 0.5
waterDifference = [merged['cdr(mm)'][i]*58500000*percentage - merged['gdf(m3/s)'][i]*86400000 for i in range(len(merged['cdr(mm)']))]
cumulativeWaterDifference = [sum(waterDifference[:i]) for i in range(len(waterDifference))]
averageCum = sum(cumulativeWaterDifference)/len(cumulativeWaterDifference)

while (averageCum >= 100 or averageCum <= -100):
    waterDifference = [merged['cdr(mm)'][i]*58500000*percentage - merged['gdf(m3/s)'][i]*86400000 for i in range(len(merged['cdr(mm)']))]
    cumulativeWaterDifference = [sum(waterDifference[:i]) for i in range(len(waterDifference))]
    averageCum = sum(cumulativeWaterDifference)/len(cumulativeWaterDifference)
    if averageCum >= 100:
        percentage -= scale
    elif averageCum <= -100:
        percentage += scale
    scale /= 2


print("Calculated percentage of water which ends up in river", percentage)

waterDifference = [merged['cdr(mm)'][i]*58500000*percentage - merged['gdf(m3/s)'][i]*86400000 for i in range(len(merged['cdr(mm)']))]
cumulativeWaterDifference = [sum(waterDifference[:i]) for i in range(len(waterDifference))]

print("\nminimum cumulative water difference:", min(cumulativeWaterDifference))
print("maximum cumulative water difference:",max(cumulativeWaterDifference))
print("mean cumulative water difference:",sum(cumulativeWaterDifference)/len(cumulativeWaterDifference))

The final calculated percentage of water which ends up in the river is 64.7%, which is fairly close to the Scotish Governments stated estimate of 73%. In the future we could look at using a more accurate value for this, or even a value which changes throughout the year to account for the weather conditions and explore why their is a difference between the two values. However, for now we will continue with the assumption that 64.7% of the rainfall ends up in the river and continue with our analysis.

In [ ]:
waterDifference = [merged['cdr(mm)'][i]*58500000*0.697 - merged['gdf(m3/s)'][i]*86400000 for i in range(len(merged['cdr(mm)']))]
cumulativeWaterDifference = [sum(waterDifference[:i]) for i in range(len(waterDifference))]

merged['water_difference'] = waterDifference
merged['cumulative_water_difference'] = cumulativeWaterDifference
merged.describe()

In [ ]:
# Plot the water difference and cumulative water difference
plt.plot(merged['date'], merged['water_difference'])
plt.plot(merged['date'], merged['cumulative_water_difference'])
plt.legend(['Water Difference', 'Cumulative Water Difference'])
plt.show()


In [ ]:
# Plot the water difference and the level against the date
plt.plot(merged['date'], merged['water_difference'] / 1000000000)
plt.plot(merged['date'], merged['avg_level'])
plt.legend(['Water Difference (in billions)', 'Level'])
plt.show()

As can be seen from the graph above, when we have a day with a large positive water difference, the water level rises shortly after. Likewise, when we have a day of negative rainfall, we see the water difference decrease as we'd expect. The next step will be to plot the differentiated water level as this will give us a better idea of the change in water level dependent upon the water difference.

In [ ]:
# Calculate the derivative of the water level
levelDerivative = [merged['avg_level'][i+1] - merged['avg_level'][i] for i in range(len(merged['avg_level'])-1)]
levelDerivative.append(0)
merged['level_derivative'] = levelDerivative

# Plot the water difference and the level derivative against the date with different axis scales
fig, ax1 = plt.subplots()
ax1.plot(merged['date'], merged['water_difference'])
ax1.set_xlabel('Date')
ax1.set_ylabel('Water Difference', color='tab:blue')
ax1.set_ylim(-1000000000, 1000000000)
ax2 = ax1.twinx()
ax2.plot(merged['date'], merged['level_derivative'], color='tab:red')
ax2.set_ylabel('Level Derivative', color='tab:red')
ax2.set_ylim(-1, 1)
plt.show()

Damn that looks good :) love it when the data looks like it's doing what you expect it to do. The reason why the water difference is divided by a billion 

In [ ]:
# Plot the level_derivative against the rainfall
merged.describe()


In [ ]:
cols = ['gdf(m3/s)', 'cdr(mm)', 'avg_level', 'water_difference', 'level_derivative']
sns.pairplot(merged[cols], height=2.5)

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(merged[cols].values)
cov_mat = np.cov(X_std.T)
plt.figure(figsize=(10, 10))
hm = sns.heatmap(cov_mat, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 15}, cmap='coolwarm', yticklabels=cols, xticklabels=cols)
plt.title('Covariance Matrix')
plt.tight_layout()
plt.show()